<a href="https://colab.research.google.com/github/EMGS-GROUP/CVtheque/blob/main/OCR_V2_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Paddle installation

In [ ]:
# !python -m pip install paddlepaddle-gpu==2.0.0 -i https://mirror.baidu.com/pypi/simple

In [ ]:
# !sudo pip install paddlepaddle

In [ ]:
# !pip install paddleocr

In [ ]:
# !pip install ArabicOcr

# Last working method

In [ ]:
# importation des bibliothèques
from paddleocr import PaddleOCR, draw_ocr
from ArabicOcr import arabicocr
from matplotlib import pyplot as plt
import math
import cv2
import os
import re

In [ ]:
### Choose your country:
print("Choose your country here : ")
country = input()
if country == "Emarates":
    lang = "en"
    _op = 'op1'
elif country == "Egypt":
    lang = "arabic"
    _op = 'op2'
elif country == "Morocco" or country == "France" or country == "England" or country == "Spain":
    lang = "en"
    _op = 'op1'
else:
    lang = "latin"
    _op = 'op1'

In [ ]:
img_paths = ["/content/attestation_urssaf_emgs_group_21012022.jpg"]

In [ ]:
# la liste qui contiendra les mots-phrases extraits
words=[]
# 2 Options : soit nous utilisons le modèle pré-entraîné pour l'arabe ou l'autre modèle pour les autres langues (mais nous n'utiliserons que l'anglais)
if _op == "op2":
    for img_path in img_paths:
        # Arabic OCR : Arabe
        results=arabicocr.arabic_ocr(img_path,'out.jpg')
        for i in range(len(results)):	
            word=results[i][1]
            words.append(word)
else:
    # Paddle OCR : Anglais
    ocr_model = PaddleOCR(lang=lang)
    for img_path in img_paths:
        results= ocr_model.ocr(img_path)
        for res in results:
            word = res[1][0]
            words.append(word) 

In [ ]:
words

# <font color='blue'>Card ID number/valid date detection</font>

# Preprocessing of (Kuwait-Qatar-UAE-Poland-Croatia-Turkey-Russia-China-India-Italy-Netherlands-Algeria-Switzerland-Belgium-Portugal-Germany-Morocco-England-French-Spain)_national_id_card
_______________________

**Preprocessing starts here**
________________________

In [ ]:
################################################################## CARD_ID_Detection #################################################################################

import re
# initialiser la variable Card_id si aucune de ces condition n'est détecter alors l'OCR n'a pas détecter Card_id_number 
card_id = "Not detected"
# J'ajoute ces variable pour les utiliser comme des verrouillages car parfois il detecte qu'il s'agit d'une carte portogaise mais les autres conditions peuvent etre vérifier donc le traitement change, alors ce verrouillage le provoque et ferme le traitement sur les autres conditions 
c = f = s = indb = t = e = ku = 0
for k in range(len(words)):
    # Testez s'il y avait "United Arab Emirates..." pour détecter qu'il s'agit d'une carte d'identité kuwait pour faire son traitement qui va détecter que le numéro mais pas de date de validité car elle n'en contient pas
    if bool(re.findall(r'STATE OF KUWAIT|STATE OFKUWAIT|STATEOF KUWAIT|STATEOFKUWAIT|KUWAIT', words[k])):
        if [id for id in words if id.isdigit() and not id.isalpha() and sum(map(str.isdigit, id))>8 and len(id) >8]:
                card_id = [id for id in words if id.isdigit() and not id.isalpha() and sum(map(str.isdigit, id))>8 and len(id) >8][-1]
                ku = 1
     # Testez s'il y avait "United Arab Emirates..." pour détecter qu'il s'agit d'une carte d'identité émarates pour faire son traitement qui va détecter que le numéro mais pas de date de validité car elle n'en contient pas
    elif bool(re.findall(r'United Arab Emirates|Emirates|EMIRATES|UNITED ARAB EMIRATES|UnitedArabEmirates|UNITEDARABEMIRATES', words[k])) and ku!=1:
        for j in range(len(words)):
            if [words[j] for i in range(len(words[j])) if words[j][i]=='-' and sum(map(str.isdigit, words[j]))>8]:
                card_id = [words[j] for i in range(len(words[j])) if words[j][i]=='-' and sum(map(str.isdigit, words[j]))>8][-1]
                e = 1
    # Testez s'il y avait "TURKiYE CUMHURIYETi KIMLiK KARTI..." pour détecter qu'il s'agit d'une carte d'identité turkey pour faire son traitement qui va détecter que le numéro mais pas de date de validité car elle n'en contient pas
    elif bool(re.findall(r'TORKiYE CUMHURIYETi KIMLiK KARTI|TORKiYE|TURKIYE|TORKIYE|REPUBLIC Of TURKEY|REPUBLICOfTURKEY|TURKEY|Türkiye|TURKiYE CUMHURiYETIKiMLiKKARTI|TURKiYECUMHURiYETIKiMLiKKARTI', words[k])) and ku!=1 and e!=1:
        if [id for id in words if id.isdigit() and not id.isalpha() and sum(map(str.isdigit, id))==11 and len(id) == 11]:
            card_id = [id for id in words if id.isdigit() and not id.isalpha() and sum(map(str.isdigit, id))==11 and len(id) == 11][-1]
            t = 1
     # Testez s'il y avait "Government of India" pour détecter qu'il s'agit d'une carte d'identité indienne pour faire son traitement qui va détecter que le numéro mais pas de date de validité car elle n'en contient pas
    elif bool(re.findall(r"Government of India|GOVERNMENTIOFINDIA|GOVERNMENTOFINDIA|GOVERNMENT OF INDIA|GovernmentofIndia|India|INDIA", words[k])) and ku!=1 and t!=1 and e!=1:
        # Détectez le numéro qui ne contient que des chiffres et parfois l'extracteur PaddleOCR ajoute de l'espace qui existe donc je l'intègre ci-dessous dans Regex
        sim = [s for s in words if re.match("^[0-9 ]+$", s)]
        if sim:
            # Retourner le résultat s'il existe
            card_id = sim[-1]
            indb = 1
    # Testez s'il y avait "PORTUGAL ..." pour détecter qu'il s'agit d'une carte d'identité portogaise 
    elif bool(re.findall(r"PORTUGAI|PORTU|PORTUG|PORTUGAL", words[k])) and ku!=1 and e!=1 and t!=1:
        # le numéro de cette carte contient des chiffre, espace puis des lettres "23487698 8 ZU9" alors Regex ici détecte ces trois plus nombre de chiffre doit > 10 pour enlever les autres fausses extractions car la date de validité est donnée également dans la meme ligne.  
        m = [idx for idx in words if bool(re.match('[A-Z0-9\s]+$', idx)) and sum(map(str.isdigit, idx))>10]
        if m:
            # ici j'ai enlever cet espace "32026230 8 ZZ6 26 072022" Date :'26 072022' et j'écarte la date pour conserver ce numéro
            card_id = m[0].replace(" ","")[:-8:1]
            # activer le verrouillage pour empêcher d'autres conditions-traitement de me donner de faux résultats
            c = 1
    # Testez s'il y avait "DNI NUM ..." pour détecter qu'il s'agit d'une carte d'identité espagnole
    elif bool(re.findall(r"DN|DNI|DNINUM|DNI NUM", words[k])) and ku!=1 and e!=1 and c!=1 and t!=1:
        # Ne cherchez que les sorties OCR qui contiennent des nombres mais qui sont positionnées juste après les expressions 'DNI NUM...' par un pas ou deux pas qui se situe juste à la fin de la carte. 
        if [i for i in range(len(words[k])) if words[k][i].isdigit()]: # Condition qui teste si on trouver ces expression - tester le remplissage de liste 
            # Enlever cette expression 'DNI NUM...' pour garder juste le numéro de la carte d'identité. 
            card_id = words[k][[i for i in range(len(words[k])) if words[k][i].isdigit()][0]:] # first step after "DNI..." lookup for card_id number
            # activer le verrouillage pour empêcher d'autres conditions-traitement de me donner de faux résultats
            s = 1
        else:
            # Après trouver l'expression "DNI..." chercher le deuxième pas s'il contient les numéro ==> Card_id number
            card_id = words[k-1]
            # activer le verrouillage pour les cartes espagnoles pour empêcher d'autres conditions-traitement de me donner de faux résultats
            s = 1
    # Pour détecter celui des cartes françaises (nouvelle version) après les expressions "DOCUMENT|DOCUMENTID" avec un pas ou deux pas ce numéro vient d'apparaitre
    elif bool(re.findall(r"DOCUMENT|DOCUMENTID|IDENTITY CARD NUMBER|IDENTITYCARDNUMBER|IDENTITYCARD NUMBER|IDENTITY CARDNUMBER", words[k])) and not bool(re.findall(r"DOCUMENTO", words[k])) and ku!=1 and t!=1 and e!=1 and c!=1:
        # Tester que après les expressions "DOCUMENT|DOCUMENTID" par un pas il y aura ce numero qui doit contenir des chiffres mais également des lettres
        if [i for i in range(len(words[k+1])) if words[k+1][i].isdigit()]:
            card_id = words[k+1]
            # activer le verrouillage pour les cartes françaises pour empêcher d'autres conditions-traitement de me donner de faux résultats
            f = 1
        else:
            # Teste du deuxième pas s'il contient des chiffres et lettres car le premier pas ne les contient pas
            card_id = words[k+2]
            # activer le verrouillage pour les cartes françaises pour empêcher d'autres conditions-traitement de me donner de faux résultats
            f = 1
    # Pour détecter celui des cartes françaises (l'ancienne version) après les expressions "CARTE|CARTENATIONALED..." avec un pas ou deux pas ce numéro vient d'apparaitre
    elif bool(re.findall(r"CARTE|CARTENATIONALED|CARTENATIONALEDIDENTITE|CARTE NATIONALEDIDENTITEN", words[k])) and ku!=1 and e!=1 and c!=1 and t!=1:
        # Après la détection de ce motif il est attaché avec le numéro de la carte d'identité.
        if [i for i in range(len(words[k])) if words[k][i].isdigit()]: #Tester la non vide de la liste qui va contenir le motif "expression dessus + Card_id"
            card_id = words[k][[i for i in range(len(words[k])) if words[k][i].isdigit()][0]:] # Extarction que du card_id
            # activer le verrouillage pour les cartes françaises pour empêcher d'autres conditions-traitement de me donner de faux résultats
            f = 1
    ############################## Celui peut-etre merger avec le premier de France card id --- à tester pour choisir un seul code ##############################################
    # Pour détecter celui des cartes françaises (nouvelle version) après les expressions "DOCUMENTIDocument|DOCUMENT|DOC|DOCU|DocumentNo" avec un pas ou deux pas ce numéro vient d'apparaitre, ce code fait comme celui vu dessus mais celui dessus s'intersse que pour les card_id digits mais celui la detecte digits et lettres majuscules
    elif bool(re.findall(r"DOCUMENTIDocument|DOCUMENT|DOC|DOCU|DocumentNo", words[k])) and ku!=1 and e!=1 and c!=1 and not bool(re.findall(r"DOCUMENTO", words[k])) and t!=1:
            #chercher des numéro et lettres majuscules
            if all(c.isdigit() or c.isupper() for c in words[k+1]):
                # les chercher dans premiere position après cette expression
                if [i for i in range(len(words[k+1])) if words[k+1][i].isdigit()]:
                        card_id = words[k+1]
                        # Verrouillage
                        f = 1
                # # les chercher dans deuxième position après cette expression
                elif [i for i in range(len(words[k+2])) if words[k+2][i].isdigit()]:
                        card_id = words[k+2]
                        # verrou
                        f = 1
    # Pour tester ceux Marocaine
    elif all(c.isdigit() or c.isupper() for c in words[k]) and ku!=1 and e!=1 and c!=1 and s!=1 and t!=1: # chercher s'il s'agit chiffres et des lettres majuscules 
        for idx in words:
            # Après les lettres détectés ne doivent pas etre > 3 (au Maroc < 3 lettres dans chaque card_id number)
            if idx.isalnum() and not idx.isalpha() and not idx.isdigit() and idx.isupper() and len(idx) > 3:
                card_id = idx
                # utiliser ici le verro français
                f = 1
    # Ici pour déteter celui de Belgium card_id qui contient des numéro et des tirets du 6 '-'
    elif [idx for idx in words if bool(re.match('[A-Z0-9\-]+$', idx)) and sum(map(str.isdigit, idx))>8] and ku!=1 and e!=1 and t!=1 and c!=1 and f!=1 and s!=1: # checker la liste
        card_id = [idx for idx in words if bool(re.match('[A-Z0-9\-]+$', idx)) and sum(map(str.isdigit, idx))>8][-1] # prendre l'element de cette liste
###### si aucune de ces conditions n'est vérifiée, celle-là sera considéré
if all(c.isdigit() or c.isupper() for c in words[0]) and ku!=1 and e!=1 and c!=1 and f!=1 and t!=1: # pour celle de Bretagne qui contient les chiffres ou mélange entre chiffres-lettres, au sommet de la liste words 
        if [i for i in range(len(words[0])) if words[0][i].isdigit()]:
            card_id = words[0] # prendre le premier comme étant card_id number
        # Pour détecter celle de China car leur card_id est sous forme des chiffres  
        elif [id for id in words if id.isdigit() and not id.isalpha() and sum(map(str.isdigit, id))>=10 and len(id) >= 10] and indb!=1:
            card_id = [id for id in words if id.isdigit() and not id.isalpha() and sum(map(str.isdigit, id))>=10 and len(idx) >= 10][-1]
        else: # si non il doit parcourir toute la liste words pour cherhcer le motif qui doit contenir un mélange de chiffres et lettres pour couvrir Italy-Germany
            res = []
            for idx in words:
                # prendre que ce mélange lettres-chiffres 
                if idx.isalnum() and not idx.isalpha() and not idx.isdigit() and sum(map(str.isalpha, idx))<3 and len(idx) > 3:
                    res.append(idx)
            if len(res) > 0:
                card_id = res[-1]

In [ ]:
################################################################## Date_Detection #################################################################################

# Date_Detection
from datetime import date
# initialisation de cette variable
valable_date = 'No date is detected'
# initialisation des listes  
m = l = []
# J'ajoute ces variable pour les utiliser comme des verrouillages car parfois il detecte qu'il s'agit d'une carte portogaise mais les autres conditions peuvent etre vérifier donc le traitement change, alors ce verrouillage le provoque et ferme le traitement sur les autres conditions
a = b = f = t = n = indob = 0
for k in range(len(words)-1):
    # traitement des date pour Pays-bas
    if bool(re.findall(r'KONINKRIJK DER NEDERLANDEN|NEDERLANDSE|KINGDOM OF THE NETHERLANDS|ROYAUME DES PAYS-BAS|NEDERLAND|KINGDOMOFTHENETHERLANDS|NEDERLANDSE|ROYAUMEDESPAYSBAS', words[k])):
        l0 = []
        for k in range(len(words)-1):
            # Dictionnaire pour convertir les mois lettre en format chiffre pour garder une seule forme pour la date 
            if re.findall(r"/JAN|/FEB|/MAR|/APR|/MAY|/JUN|/JUL|/AUG|/SEP|/OCT|/NOV|/DEC", words[k]): # localiser où ces dates existe 
                l0.append(words[k])
        dic = {"JAN" : "01", "FEB" : "02","MAR" : "03","APR" : "04","MAY" : "05","JUN" : "06","JUL" : "07","AUG" : "08","SEP" : "09","OCT" : "10","NOV" : "11","DEC" : "12",}
        l1 = []
        l2 = []
        # la conversion des ces dates par l'équivalent de chaque mois en chiffres 'JAN ==> 01 ....'
        for k in dic.keys():
            for j in l0:
                if k in j:
                    l1.append(str(dic[k]).join(re.findall(r'\d+', j)))
                    l2.append(str(dic[k]).join(re.findall(r'\d+', j))[4:])
        # retourner la date par un découpage ---jour---mois---année
        valable_date = re.sub('\D', '', l1[l2.index(max(l2))])[:2]+'.' + re.sub('\D', '', l1[l2.index(max(l2))])[2:4] + "." + re.sub('\D', '', l1[l2.index(max(l2))])[4:]
        # activation du Verrou
        n = 1
    # détecter celle de Portugal
    elif bool(re.findall(r"PORTUGAI|PORTU|PORTUG|PORTUGAL", words[k])) and n!=1:
        # puisque card_id et date existe en meme ligne alors je garde que celle mélanger chiffre numéro pour l'employer dans le traitement qui va venir en bas en Bloc_P.
        m = [idx for idx in words if bool(re.match('[A-Z0-9\s]+$', idx)) and sum(map(str.isdigit, idx))>10]
        # Verrou
        a = 1
    # détecter celle de Suisse qui est dans cette forme : 01-01-19
    elif bool(re.findall(r"Suisse|SCHWEIZERISCHE|SWISS CONFEDERATION|SWISSCONFEDERATION|SWISS", words[k])) and n!=1:
        for k in range(len(words)-1):
            # detecter par cette condition les premiers deux chiffres comme jours < 31 puis mois < 12 puis années + l'OCR donne ici deux dates : délivrence + expiration alors la condition va checker aussi les deux date doivent venir consécutivement
            if words[k].isdigit() and sum(map(str.isdigit, words[k]))==6 and words[k][:2]<="31" and words[k][2:4]<="12" and words[k+1].isdigit() and sum(map(str.isdigit, words[k+1]))==6 and words[k+1][:2]<="31" and words[k+1][2:4]<="12":
                l.extend([words[k],words[k+1]]) # l'ajout à cette liste
            if l:
                # découpage de ces dates puis les prendre sous la forme usuel des dates qu'on adopte
                valable_date = [x for x in l if x[4:6] == max(l[0][4:6],l[1][4:6])][-1][:2]+"."+[x for x in l if x[4:6] == max(l[0][4:6],l[1][4:6])][-1][2:4]+"."+"20"+[x for x in l if x[4:6] == max(l[0][4:6],l[1][4:6])][-1][4:6]
                # Verrou Suisse
                b = 1
    # Pour les cartes françaises (version ancienne) l'OCR me donne le résutat suivant {"Carte valable jusqu'au 21.06.1998"}
    # cette condition trouvera cette ligne
    elif [i for i in range(len(words[k])) if words[k][i].isdigit()] and bool(re.findall("Carte valable jusqu'au|valable jusqu'au|jusqu'au|jusquau", words[k])) and a!=1 and b!=1 and n!=1:
        v = words[k][[i for i in range(len(words[k])) if words[k][i].isdigit()][0]:] # extraire la partie qui contient des digits "21.06.1998"
        valable_date = re.sub('\D', '', v)[:2]+'.' + re.sub('\D', '', v)[2:4] + "." + re.sub('\D', '', v)[4:]
        # Verrou français
        f = 1
    # Codition pour Belgium afin d'activer le Verrou pour la condition du Bloc_B
    elif bool(re.findall(r"BELGIE|BELGIQUE|BELGIEN|BELGIUM", words[k])):
        t = 1
if m and a==1 and b!=1 and f!=1 and n!=1: ###################### Bloc_P de traitement pour portugal ######################
    valable_date = m[0].replace(" ","").replace(m[0].replace(" ","")[:-8:1],"")[:2]+"."+m[0].replace(" ","").replace(m[0].replace(" ","")[:-8:1],"")[2:4]+"."+m[0].replace(" ","").replace(m[0].replace(" ","")[:-8:1],"")[4:]
# si aucun des ces verrous est activer cette condition s'active, un peu plus une condition générale qui verifie la date de forme usuelle
elif a!=1 and b!=1 and f!=1 and n!=1:
    # liste qui va contenir la détection des 4 chiffres mais il doivent commencer par "1" ou "2" et que ce motif ne soit pas supérieur à la date actuelle + 10 ans pour éviter de prendre "9765 ..."  
    _imdate = [k for k in re.findall("[0-9]{4}", " ".join(words)) if k.startswith('1') or k.startswith('2') and k < str(date.today().year+10)]
    if _imdate:
        # si les dates sont extraits {date of birth : 1991- date of issue : 2007- expiry date : 2017} alors je prend le max 2017 qui sera la date de validité
        im_date = max(_imdate)
        for i in words:
            if im_date in i:
                n = i
                break
        # je prend le reste des nombres (8 chiffres) qui accompagne celui-la : "12102017" qui se traduit par "12.10.2017"
        new_l = ''.join(e for e in n if e.isalnum())[-8:]
        if [words.index(i) for i in words if im_date in i]:
            # ici pour enlver le cas de la carte d'Inde si je détecte l'un des expressions suivant "/DOB|/dob|/Year of Birth|/YearofBirth" j'utilisera un Verrouillage pour ne pas prendre la date de naissance et la considèrer comme date de validité
            if bool(re.findall(r"/DOB|/dob|/Year of Birth|/YearofBirth", words[[words.index(i) for i in words if im_date in i][-1]])):
                indob = 1 # Indian Verrou
        if t == 1 and indob != 1: ###################### Bloc_B de traitement pour Belgium ######################
            valable_date = re.sub('\D', '', new_l)[:2]+'.' + re.sub('\D', '', new_l)[2:4] + "." + re.sub('\D', '', new_l)[4:]
        # si il s'agit des chiifres normaux (date de format usuelle) alors je prend ces chiffres {2 chiffres :jours} - {2 chiffres : mois} - {4 chiffres : année}
        elif sum(map(str.isdigit, new_l))>=6 and t!=1 and indob != 1:
            # si les 4 chiffres d'année détectés sont à l'l'extrémité des 8 chiffres detetcter alors c'est un traitement "découpage" normal : jour.mois.année
            if new_l.find(im_date) == len(new_l)-4:
                valable_date = re.sub('\D', '', new_l)[:2]+'.' + re.sub('\D', '', new_l)[2:4] + "." + re.sub('\D', '', new_l)[4:]
            # Dans le cas échéant le découpage change ###### Cas des dates pour carte d'Algérie  année.mois.jour
            else: 
                valable_date = re.sub('\D', '', new_l)[:4]+'.' + re.sub('\D', '', new_l)[4:6] + "." + re.sub('\D', '', new_l)[6:]

In [ ]:
print("date de validité de la CIN : {} || Numéro de la CIN : {}".format(valable_date, card_id))

_______________________

**Preprocessing ends here**
________________________

# <font color='orange'>Passport number/valid date detection</font>

# Pre-processing of passports for extraction (valid date, passport N°)

In [ ]:
img_paths = ["/content/testUSPassport.png"]
# la liste qui contiendra les mots-phrases extraits
words=[]
# 2 Options : soit nous utilisons le modèle pré-entraîné pour l'arabe ou l'autre modèle pour les autres langues (mais nous n'utiliserons que l'anglais)
if _op == "op2":
    for img_path in img_paths:
        # Arabic OCR : Arabe
        results=arabicocr.arabic_ocr(img_path,'out.jpg')
        for i in range(len(results)):	
            word=results[i][1]
            words.append(word)
else:
    # Paddle OCR : Anglais
    ocr_model = PaddleOCR(lang=lang)
    for img_path in img_paths:
        results= ocr_model.ocr(img_path)
        for res in results:
            word = res[1][0]
            words.append(word) 

In [ ]:
words

In [ ]:
################################################################## Passport_No_Detection #################################################################################

import re
words1 = words[:math.ceil(len(words)/2)]
# initialiser la variable Card_id si aucune de ces condition n'est détecter alors l'OCR n'a pas détecter Card_id_number 
card_id = "Not detected"
# J'ajoute ces variable pour les utiliser comme des verrouillages car parfois il detecte qu'il s'agit d'une carte portogaise mais les autres conditions peuvent etre vérifier donc le traitement change, alors ce verrouillage le provoque et ferme le traitement sur les autres conditions 
c = f = s = indb = t = e = ku = 0
for k in range(len(words1)):
    # Testez s'il y avait "United Arab Emirates..." pour détecter qu'il s'agit d'une carte d'identité kuwait pour faire son traitement qui va détecter que le numéro mais pas de date de validité car elle n'en contient pas
    if bool(re.findall(r'STATE OF KUWAIT|STATE OFKUWAIT|STATEOF KUWAIT|STATEOFKUWAIT|KUWAIT', words1[k])):
        if [id for id in words1 if id.isdigit() and not id.isalpha() and sum(map(str.isdigit, id))>8 and len(id) >8]:
                card_id = [id for id in words1 if id.isdigit() and not id.isalpha() and sum(map(str.isdigit, id))>8 and len(id) >8][-1]
                ku = 1
     # Testez s'il y avait "United Arab Emirates..." pour détecter qu'il s'agit d'une carte d'identité émarates pour faire son traitement qui va détecter que le numéro mais pas de date de validité car elle n'en contient pas
    elif bool(re.findall(r'United Arab Emirates|Emirates|EMIRATES|UNITED ARAB EMIRATES|UnitedArabEmirates|UNITEDARABEMIRATES', words1[k])) and ku!=1:
        for j in range(len(words1)):
            if [words1[j] for i in range(len(words1[j])) if words1[j][i]=='-' and sum(map(str.isdigit, words1[j]))>8]:
                card_id = [words1[j] for i in range(len(words1[j])) if words1[j][i]=='-' and sum(map(str.isdigit, words1[j]))>8][-1]
                e = 1
    # Testez s'il y avait "TURKiYE CUMHURIYETi KIMLiK KARTI..." pour détecter qu'il s'agit d'une carte d'identité turkey pour faire son traitement qui va détecter que le numéro mais pas de date de validité car elle n'en contient pas
    elif bool(re.findall(r'TORKiYE CUMHURIYETi KIMLiK KARTI|TORKiYE|TURKIYE|TORKIYE|REPUBLIC Of TURKEY|REPUBLICOfTURKEY|TURKEY|Türkiye|TURKiYE CUMHURiYETIKiMLiKKARTI|TURKiYECUMHURiYETIKiMLiKKARTI', words1[k])) and ku!=1 and e!=1:
        if [id for id in words1 if id.isdigit() and not id.isalpha() and sum(map(str.isdigit, id))==11 and len(id) == 11]:
            card_id = [id for id in words1 if id.isdigit() and not id.isalpha() and sum(map(str.isdigit, id))==11 and len(id) == 11][-1]
            t = 1
     # Testez s'il y avait "Government of India" pour détecter qu'il s'agit d'une carte d'identité indienne pour faire son traitement qui va détecter que le numéro mais pas de date de validité car elle n'en contient pas
    elif bool(re.findall(r"Government of India|GOVERNMENTIOFINDIA|GOVERNMENTOFINDIA|GOVERNMENT OF INDIA|GovernmentofIndia|India|INDIA", words1[k])) and ku!=1 and t!=1 and e!=1:
        # Détectez le numéro qui ne contient que des chiffres et parfois l'extracteur PaddleOCR ajoute de l'espace qui existe donc je l'intègre ci-dessous dans Regex
        sim = [s for s in words1 if re.match("^[0-9 ]+$", s)]
        if sim:
            # Retourner le résultat s'il existe
            card_id = sim[-1]
            indb = 1
    # Testez s'il y avait "PORTUGAL ..." pour détecter qu'il s'agit d'une carte d'identité portogaise 
    elif bool(re.findall(r"PORTUGAI|PORTU|PORTUG|PORTUGAL", words1[k])) and ku!=1 and e!=1 and t!=1:
        # le numéro de cette carte contient des chiffre, espace puis des lettres "23487698 8 ZU9" alors Regex ici détecte ces trois plus nombre de chiffre doit > 10 pour enlever les autres fausses extractions car la date de validité est donnée également dans la meme ligne.  
        m = [idx for idx in words1 if bool(re.match('[A-Z0-9\s]+$', idx)) and sum(map(str.isdigit, idx))>10]
        if m:
            # ici j'ai enlever cet espace "32026230 8 ZZ6 26 072022" Date :'26 072022' et j'écarte la date pour conserver ce numéro
            card_id = m[0].replace(" ","")[:-8:1]
            # activer le verrouillage pour empêcher d'autres conditions-traitement de me donner de faux résultats
            c = 1
    # Testez s'il y avait "DNI NUM ..." pour détecter qu'il s'agit d'une carte d'identité espagnole
    elif bool(re.findall(r"DN|DNI|DNINUM|DNI NUM", words1[k])) and ku!=1 and e!=1 and c!=1 and t!=1:
        # Ne cherchez que les sorties OCR qui contiennent des nombres mais qui sont positionnées juste après les expressions 'DNI NUM...' par un pas ou deux pas qui se situe juste à la fin de la carte. 
        if [i for i in range(len(words1[k])) if words1[k][i].isdigit()]: # Condition qui teste si on trouver ces expression - tester le remplissage de liste 
            # Enlever cette expression 'DNI NUM...' pour garder juste le numéro de la carte d'identité. 
            card_id = words1[k][[i for i in range(len(words1[k])) if words1[k][i].isdigit()][0]:] # first step after "DNI..." lookup for card_id number
            # activer le verrouillage pour empêcher d'autres conditions-traitement de me donner de faux résultats
            s = 1
        else:
            # Après trouver l'expression "DNI..." chercher le deuxième pas s'il contient les numéro ==> Card_id number
            card_id = words1[k-1]
            # activer le verrouillage pour les cartes espagnoles pour empêcher d'autres conditions-traitement de me donner de faux résultats
            s = 1
    # Pour détecter celui des cartes françaises (nouvelle version) après les expressions "DOCUMENT|DOCUMENTID" avec un pas ou deux pas ce numéro vient d'apparaitre
    elif bool(re.findall(r"DOCUMENT|DOCUMENTID|IDENTITY CARD NUMBER|IDENTITYCARDNUMBER|IDENTITYCARD NUMBER|IDENTITY CARDNUMBER", words1[k])) and not bool(re.findall(r"DOCUMENTO", words1[k])) and ku!=1 and t!=1 and e!=1 and c!=1:
        # Tester que après les expressions "DOCUMENT|DOCUMENTID" par un pas il y aura ce numero qui doit contenir des chiffres mais également des lettres
        if [i for i in range(len(words1[k+1])) if words1[k+1][i].isdigit()]:
            card_id = words1[k+1]
            # activer le verrouillage pour les cartes françaises pour empêcher d'autres conditions-traitement de me donner de faux résultats
            f = 1
        else:
            # Teste du deuxième pas s'il contient des chiffres et lettres car le premier pas ne les contient pas
            card_id = words1[k+2]
            # activer le verrouillage pour les cartes françaises pour empêcher d'autres conditions-traitement de me donner de faux résultats
            f = 1
    # Pour détecter celui des cartes françaises (l'ancienne version) après les expressions "CARTE|CARTENATIONALED..." avec un pas ou deux pas ce numéro vient d'apparaitre
    elif bool(re.findall(r"CARTE|CARTENATIONALED|CARTENATIONALEDIDENTITE|CARTE NATIONALEDIDENTITEN", words1[k])) and ku!=1 and e!=1 and c!=1 and t!=1:
        # Après la détection de ce motif il est attaché avec le numéro de la carte d'identité.
        if [i for i in range(len(words1[k])) if words1[k][i].isdigit()]: #Tester la non vide de la liste qui va contenir le motif "expression dessus + Card_id"
            card_id = words1[k][[i for i in range(len(words1[k])) if words1[k][i].isdigit()][0]:] # Extarction que du card_id
            # activer le verrouillage pour les cartes françaises pour empêcher d'autres conditions-traitement de me donner de faux résultats
            f = 1
    ############################## Celui peut-etre merger avec le premier de France card id --- à tester pour choisir un seul code ##############################################
    # Pour détecter celui des cartes françaises (nouvelle version) après les expressions "DOCUMENTIDocument|DOCUMENT|DOC|DOCU|DocumentNo" avec un pas ou deux pas ce numéro vient d'apparaitre, ce code fait comme celui vu dessus mais celui dessus s'intersse que pour les card_id digits mais celui la detecte digits et lettres majuscules
    elif bool(re.findall(r"DOCUMENTIDocument|DOCUMENT|DOC|DOCU|DocumentNo", words1[k])) and ku!=1 and e!=1 and c!=1 and not bool(re.findall(r"DOCUMENTO", words1[k])) and t!=1:
            #chercher des numéro et lettres majuscules
            if all(c.isdigit() or c.isupper() for c in words1[k+1]):
                # les chercher dans premiere position après cette expression
                if [i for i in range(len(words1[k+1])) if words1[k+1][i].isdigit()]:
                        card_id = words1[k+1]
                        # Verrouillage
                        f = 1
                # # les chercher dans deuxième position après cette expression
                elif [i for i in range(len(words1[k+2])) if words1[k+2][i].isdigit()]:
                        card_id = words1[k+2]
                        # verrou
                        f = 1
    # Pour tester ceux Marocaine
    elif all(c.isdigit() or c.isupper() for c in words1[k]) and ku!=1 and e!=1 and c!=1 and s!=1 and t!=1: # chercher s'il s'agit chiffres et des lettres majuscules 
        for idx in words1:
            # Après les lettres détectés ne doivent pas etre > 3 (au Maroc < 3 lettres dans chaque card_id number)
            if idx.isalnum() and not idx.isalpha() and not idx.isdigit() and idx.isupper() and len(idx) > 3:
                card_id = idx
                # utiliser ici le verro français
                f = 1
    # Ici pour déteter celui de Belgium card_id qui contient des numéro et des tirets du 6 '-'
    elif [idx for idx in words1 if bool(re.match('[A-Z0-9\-]+$', idx)) and sum(map(str.isdigit, idx))>8] and ku!=1 and e!=1 and t!=1 and c!=1 and f!=1 and s!=1: # checker la liste
        card_id = [idx for idx in words1 if bool(re.match('[A-Z0-9\-]+$', idx)) and sum(map(str.isdigit, idx))>8][-1] # prendre l'element de cette liste
###### si aucune de ces conditions n'est vérifiée, celle-là sera considéré
if all(c.isdigit() or c.isupper() for c in words1[0]) and ku!=1 and e!=1 and c!=1 and f!=1 and t!=1: # pour celle de Bretagne qui contient les chiffres ou mélange entre chiffres-lettres, au sommet de la liste words 
        if [i for i in range(len(words1[0])) if words1[0][i].isdigit()]:
            card_id = words1[0] # prendre le premier comme étant card_id number
        # Pour détecter celle de China car leur card_id est sous forme des chiffres  
        elif [id for id in words1 if id.isdigit() and not id.isalpha() and sum(map(str.isdigit, id))>=10 and len(id) >= 10] and indb!=1:
            card_id = [id for id in words1 if id.isdigit() and not id.isalpha() and sum(map(str.isdigit, id))>=10 and len(idx) >= 10][-1]
        else: # si non il doit parcourir toute la liste words pour cherhcer le motif qui doit contenir un mélange de chiffres et lettres pour couvrir Italy-Germany
            res = []
            for idx in words1:
                # prendre que ce mélange lettres-chiffres 
                if idx.isalnum() and not idx.isalpha() and not idx.isdigit() and sum(map(str.isalpha, idx))<3 and len(idx) > 3:
                    res.append(idx)
            if len(res) > 0:
                card_id = res[-1]

In [ ]:
from datetime import date
valable_date = "No date is detected"
l0 = []
_imdate = [k for k in re.findall("[0-9]{4}", " ".join(words)) if k.startswith('1') or k.startswith('2') and k < str(date.today().year+10)]
if _imdate:
    # si les dates sont extraits {date of birth : 1991- date of issue : 2007- expiry date : 2017} alors je prend le max 2017 qui sera la date de validité
    im_date = max(_imdate)
    for i in words:
        if im_date in i:
            n = i
            break
    # je prend le reste des nombres (8 chiffres) qui accompagne celui-la : "12102017" qui se traduit par "12.10.2017"
    new_l = ''.join(e for e in n if e.isalnum())[-8:]
    if new_l.isdigit():
        if [words[k] for k in [words.index(i) for i in words if im_date in i] if len(words[k])==8]:
            new_l = [words[k] for k in [words.index(i) for i in words if im_date in i] if len(words[k])==8][-1]
        valable_date = re.sub('\D', '', new_l)[:2]+'.' + re.sub('\D', '', new_l)[2:4] + "." + re.sub('\D', '', new_l)[4:]
    else:
        lo = []
        indices = []
        for k in words:
            lo.append(re.findall(r"JAN|FEB|MAR|APR|MAY|JUN|JUL|AUG|SEP|OCT|NOV|DEC|", k))
        for m in words:
            for n in [j for j in sum([item for item in lo if not all(len(ele) == 0 for ele in item)],[]) if len(j)!=0]:
                if n in m:
                    indices.append(words.index(m))
        indices = list(set(indices))
        if indices:
            line2 = [words[i] for i in indices][[words[i] for i in indices].index(words[max(indices)])][[words[i] for i in indices][[words[i] for i in indices].index(words[max(indices)])].rfind("/")+[words[i] for i in indices][[words[i] for i in indices].index(words[max(indices)])].rfind("/")-1:]
        temp1 = re.findall(r'\d+', line2)
        res2 = list(map(int, temp1))[-1]
        if len(str(res2)) == 2:
            year = str(20)+str(res2)
        else:
            year = str(res2)
        dic = {"JAN" : "01", "FEB" : "02","MAR" : "03","APR" : "04","MAY" : "05","JUN" : "06","JUL" : "07","AUG" : "08","SEP" : "09","OCT" : "10","NOV" : "11","DEC" : "12",}
        l1 = []
        l2 = []
                # la conversion des ces dates par l'équivalent de chaque mois en chiffres 'JAN ==> 01 ....'
        for k in dic.keys():
            if k in [j for j in sum([item for item in lo if not all(len(ele) == 0 for ele in item)],[]) if len(j)!=0]:
                if k in [j for j in sum([item for item in lo if not all(len(ele) == 0 for ele in item)],[]) if len(j)!=0][-1]:
                    month = dic[k]
        if indices:
            day = [words[i] for i in indices][[words[i] for i in indices].index(words[max(indices)])][:2]
        valable_date = day+"."+month+'.'+year

In [ ]:
print("date de validité du Passeport : {} || Numéro du Passeport : {}".format(valable_date, card_id))

# <font color='red'>Administrative documents-France</font>

# KBIS number & valid date extraction

In [ ]:
# !pip install pymupdf

In [ ]:
import fitz
import re
doc = fitz.open("/content/Fr/emgs_group_kbis_794442426_26012022.pdf")
text = ""
for k in range(doc.page_count):
    text += doc.load_page(k).get_text('text')
kbis_list = text.split('\n')

for k in range(len(kbis_list)):
    if bool(re.findall(r"numéro|Numéro", kbis_list[k])):
         if [i for i in range(len(kbis_list[k+1])) if kbis_list[k+1][i].isdigit()] and "R.C.S" in kbis_list[k+1]: 
            kbis_number = kbis_list[k+1][:[i for i in range(len(kbis_list[k+1])) if kbis_list[k+1][i].isdigit()][-1]+1]

lo = []
kbis_extrait_date = ""
for k in kbis_list:
    lo.append(re.findall(r"janvier|février|mars|avril|mai|juin|juillet|août|septembre|octobre|novembre|decembre|", k))
for i in range(len(lo)):
    for j in range(len(lo[i])):
        if len(lo[i][j]) != 0 and re.findall(r"Extrait au|à jour au", kbis_list[i]):
            kbis_extrait_date = kbis_list[i][[i for i, c in enumerate(kbis_list[i]) if c.isdigit()][0]:]
dic = {"janvier" : "1", "février" : "2","mars" : "3","avril" : "4","mai" : "5","juin" : "6","juillet" : "7","août" : "8","septembre" : "9","octobre" : "10","novembre" : "11","décembre" : "12",}
for k in dic.keys():
    if k in kbis_extrait_date:
        ks = kbis_extrait_date.replace(k, dic[k])
kbis_valide_date = kbis_extrait_date.replace(kbis_extrait_date.split(" ")[1],[k for k, v in dic.items() if v == str(int(ks.split(" ")[1])+6)][-1])
print("Numéro de kbis : {} || date d'extrait kbis : {} || date de validité du kbis : {}".format(kbis_number,kbis_extrait_date,kbis_valide_date))

# URSSAF security code extraction

In [ ]:
# !pip install pdf2image
# !apt-get install poppler-utils

In [ ]:
from pdf2image import convert_from_path
import fitz
import re
file_path_ = "/content/Fr/attestation_urssaf_emgs_group_21012022.pdf"
doc = fitz.open(file_path_)
text = ""
for k in range(doc.page_count):
    text += doc.load_page(k).get_text('text')
urssaf_list = text.split('\n')
images = convert_from_path(file_path_)
pdf_name = file_path_[:-4]
image_name = pdf_name
for i in range(len(images)):
    images[i].save(image_name + '.jpg', 'JPEG')

img_path = image_name +'.jpg'
words = []
ocr_model = PaddleOCR(lang=lang)
results= ocr_model.ocr(img_path)
for res in results:
    word = res[1][0]
    words.append(word) 
if [idx for idx in words if idx.isalnum() and not idx.isalpha() and not idx.isdigit() and idx.isupper() and len(idx) > 6]:
    security_code = [idx for idx in words if idx.isalnum() and not idx.isalpha() and not idx.isdigit() and idx.isupper() and len(idx) > 6][-1]
print("\n\n\nVotre code de sécurité pour URSSAF est : ", security_code)

# Assurence expiry date

In [ ]:
import fitz
import re
import time
from datetime import datetime
doc = fitz.open("/content/Fr/emgs_group_attestation_assurance_2022.pdf")
text = ""
for k in range(doc.page_count):
    text += doc.load_page(k).get_text('text')
assurance_list = text.split('\n')
for k in range(len(assurance_list)):
    if bool(re.findall(r"Valable|valable", assurance_list[k])) and len(re.findall(r'\d\d\/\d{2}\/\d{4}', assurance_list[k])) == 2:
        assurance_expiry_date = max(re.findall(r'\d\d\/\d{2}\/\d{4}', assurance_list[k]))
current_date = datetime.now().strftime("%d/%m/%Y")
if time.strptime(assurance_expiry_date,"%d/%m/%Y") > time.strptime(current_date,"%d/%m/%Y"):
    print("La date d'expiration de votre assurance est", assurance_expiry_date)
else:
    print("Vous devez renouveler votre assurance")

# Fiscality preprocessing

In [ ]:
import fitz
import re
from dateutil.relativedelta import relativedelta
import time
from datetime import datetime
doc = fitz.open("/content/Fr/attestation_regularite_fiscale_emgs_group_24012022.pdf")
text = ""
for k in range(doc.page_count):
    text += doc.load_page(k).get_text('text')
fiscality_list = text.split('\n')
for k in range(len(fiscality_list)):
    if bool(re.findall(r'\d\d\/\d{2}\/\d{4}', fiscality_list[k])):
        fiscality_date = re.findall(r'\d\d\/\d{2}\/\d{4}', fiscality_list[k])[-1]
dtObj1 = datetime.strptime(fiscality_date, '%d/%m/%Y')
valid_fiscality_until_ = dtObj1 + relativedelta(years=1) 
valid_fiscality_until = valid_fiscality_until_.strftime('%d/%m/%Y')
print("Votre fiscalité est valide jusqu'au : ",valid_fiscality_until)

# <font color='red'>Administrative documents-England</font>

In [ ]:
from pdf2image import convert_from_path
from datetime import datetime
from dateutil.relativedelta import relativedelta
import fitz
import re

file_path_ = "/content/Eng/Companies House_Confirmation Statement_15022021_EMGS CONSULTING LTD.pdf"
doc = fitz.open(file_path_)
text = ""
for k in range(doc.page_count):
    text += doc.load_page(k).get_text('text')
urssaf_list = text.split('\n')
images = convert_from_path(file_path_)
pdf_name = file_path_[:-4]
image_name = pdf_name
for i in range(len(images)):
    images[i].save(image_name + '.jpg', 'JPEG')
    break

img_path = image_name +'.jpg'
words_ = []
ocr_model = PaddleOCR(lang=lang)
results= ocr_model.ocr(img_path)
for res in results:
    word = res[1][0]
    words_.append(word)
for k in range(len(words_)):
    if bool(re.findall(r"number|Number", words_[k])):
        if [i for i in range(len(words_[k+1])) if words_[k+1][i].isdigit()]: 
            kbis_en = words_[k+1][:[i for i in range(len(words_[k+1])) if words_[k+1][i].isdigit()][-1]+1]
    if bool(re.findall(r'\d\d\/\d{2}\/\d{4}', words_[k])) and "Received" in words_[k]:
        date_kbis_en = words_[k][[i for i in range(len(words_[k])) if words_[k][i].isdigit()][0]:]
dtObj = datetime.strptime(date_kbis_en, '%d/%m/%Y')
en_kbis_valid_date_ = dtObj + relativedelta(months=6)
en_kbis_valid_date = en_kbis_valid_date_.strftime('%d/%m/%Y')
print("\n\nYour kbis can be used until : ",en_kbis_valid_date)